In [1]:
import numpy as np
from bayeso import covariance
from bayeso.gp import gp
from bayeso.utils import utils_covariance
from bayeso.utils import utils_plotting
from bayeso.utils import utils_common
from bayeso import acquisition
import numpy as np

from bayeso import bo
from bayeso.wrappers import wrappers_bo
from bayeso.utils import utils_plotting


In [2]:
import torch
from torch import nn,optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import Compose, ToTensor, RandomCrop, RandomRotation, Normalize
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
def hyper_parameter_tunning(learning_rate,hidden_layer_size,epoch_num, batch_num, early_stop_thr,train_loader,val_loader):
    net = nn.Sequential(
    nn.Linear(28*28,hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size,10)
    ).to(device)
    optimizer = optim.SGD(net.parameters(),lr=learning_rate,weight_decay=0)
    loss_function = nn.CrossEntropyLoss()
    
    epoch_list=[]
    val_accuracy_list=[]
    best_val=0
    state=0
    total_num = batch_num
    
    net.train()
    
    for epoch in range(epoch_num):
        for i,(train_x,train_y) in enumerate(train_loader):
            train_x = train_x.view(-1, 28 * 28).to(device)
            train_y = train_y.to(device)
            y_pred = net(train_x)
            loss = loss_function(y_pred,train_y.view(-1))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        epoch_list.append(epoch+1)
        
        net.eval()
        for j,(val_x,val_y) in enumerate(val_loader):
            if j>0:
                break
            val_x = val_x.view(-1, 28 * 28).float().to(device)
            val_y = val_y.to(device)
            y_pred = net(val_x)
            _,y_class = torch.max(y_pred, 1)
            correct_num = (val_y == y_class).sum()
            val_accuracy = correct_num.cpu().numpy() / total_num *100
            val_accuracy_list.append(val_accuracy)
            
            if val_accuracy >best_val:
                state=0
                best_val=val_accuracy
            else:
                state+=1
        if state>=early_stop_thr:
            break

    return best_val

In [7]:
#num_points = 1000
str_cov = 'se'
#num_iter = 20
#num_ts = 5

list_Y_min = []

"""
bounds = np.array([[0,10],[0,10]])  ##define 필요

def fun_target(X): ##define 필요
    return X[0]+X[1]
"""
bounds = np.array([[0.01,0.05],[128,512]])  ##define 필요

def fun_target(X): ##define 필요
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())
    mnist_train, mnist_val = torch.utils.data.random_split(mnist_trainset, [int(len(mnist_trainset)/6*5), int(len(mnist_trainset)/6)])
    batch_num=512
    train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_num,shuffle=True, num_workers=2, drop_last = True)
    val_loader = torch.utils.data.DataLoader(mnist_val, batch_size=batch_num,shuffle=True, num_workers=2, drop_last = True)
    
    
    return hyper_parameter_tunning(int(X[0]),int(X[1]), 512, 50,10,train_loader,val_loader)


cur_regret_list =[]
X_list =[]
Y_list =[]
# str_cov_list = ['se', 'matern32', 'matern52']
str_cov_list = ['se']

for str_cov in str_cov_list:
    #num_bo = 3
    num_bo = 1
    #str_cov = 'se'
    num_iter = 20
    num_init = 3

    model_bo = bo.BO(bounds, debug=False,str_cov = str_cov)
    list_Y = []
    list_time = []

    for ind_bo in range(0, num_bo):
        print('BO Round', ind_bo + 1)
        #X_final, Y_final, time_final, _, _ = wrappers_bo.run_single_round(model_bo, fun_target, num_init, num_iter,str_initial_method_bo='uniform', str_sampling_method_ao='uniform', num_samples_ao=100,seed=42 * ind_bo)
        X_final, Y_final, time_final, _, _ = wrappers_bo.run_single_round(model_bo, fun_target, num_init, num_iter,str_initial_method_bo='sobol', str_sampling_method_ao='sobol', num_samples_ao=100,seed=42 * ind_bo)

        list_Y.append(Y_final)
        list_time.append(time_final)

    arr_Y = np.array(list_Y)
    arr_time = np.array(list_time)

    X_list.append(X_final)
    Y_list.append(Y_final)

    arr_Y = np.expand_dims(np.squeeze(arr_Y), axis=0)
    arr_time = np.expand_dims(arr_time, axis=0)
    
    """num_bo>1 때 plot 그려줌
    utils_plotting.plot_minimum_vs_iter(arr_Y, [str_fun], num_init, True,
        use_tex=True,
        str_x_axis=r'\textrm{Iteration}',
        str_y_axis=r'\textrm{Mininum function value}')
    utils_plotting.plot_minimum_vs_time(arr_time, arr_Y, [str_fun], num_init, True,
        use_tex=True,
        str_x_axis=r'\textrm{Time (sec.)}',
        str_y_axis=r'\textrm{Mininum function value}')
    regret = [[np.min(arr_Y[0,k,:(i+1)])-np.min(arr_Y) for i in range(arr_Y.shape[2])] for k in range(arr_Y.shape[1])]
    regret = np.array(regret)
    sum_regret = np.sum(regret)
    cur_regret_list.append(sum_regret)
    """


#mu_, sigma_, Sigma_ = gp.predict_with_optimized_hyps(X_final, Y_final, X_test, str_cov=str_cov)

[INFO-wrappers_bo-05/27/2021-10:30:17] range_X:
[[0.010, 0.050],
[128.000, 512.000]]
[INFO-wrappers_bo-05/27/2021-10:30:17] str_cov: se
[INFO-wrappers_bo-05/27/2021-10:30:17] str_acq: ei
[INFO-wrappers_bo-05/27/2021-10:30:17] str_optimizer_method_gp: BFGS
[INFO-wrappers_bo-05/27/2021-10:30:17] str_optimizer_method_bo: L-BFGS-B
[INFO-wrappers_bo-05/27/2021-10:30:17] str_modelselection_method: ml
[INFO-wrappers_bo-05/27/2021-10:30:17] num_init: 3
[INFO-wrappers_bo-05/27/2021-10:30:17] num_iter: 20
[INFO-wrappers_bo-05/27/2021-10:30:17] str_initial_method_bo: sobol
[INFO-wrappers_bo-05/27/2021-10:30:17] str_sampling_method_ao: sobol
[INFO-wrappers_bo-05/27/2021-10:30:17] num_samples_ao: 100
[INFO-wrappers_bo-05/27/2021-10:30:17] str_mlm_method: regular
[INFO-wrappers_bo-05/27/2021-10:30:17] seed: 0


BO Round 1


[INFO-wrappers_bo-05/27/2021-10:33:56] Iteration 1
[INFO-wrappers_bo-05/27/2021-10:35:06] Iteration 2
[INFO-wrappers_bo-05/27/2021-10:36:50] Iteration 3
[INFO-wrappers_bo-05/27/2021-10:37:41] Iteration 4
[INFO-wrappers_bo-05/27/2021-10:39:01] Iteration 5
[INFO-wrappers_bo-05/27/2021-10:39:59] Iteration 6
[INFO-wrappers_bo-05/27/2021-10:41:05] Iteration 7
[INFO-wrappers_bo-05/27/2021-10:41:55] Iteration 8
[INFO-wrappers_bo-05/27/2021-10:42:42] Iteration 9
[INFO-wrappers_bo-05/27/2021-10:43:44] Iteration 10
[INFO-wrappers_bo-05/27/2021-10:44:50] Iteration 11
[INFO-wrappers_bo-05/27/2021-10:46:23] Iteration 12
[INFO-wrappers_bo-05/27/2021-10:47:29] Iteration 13
[INFO-wrappers_bo-05/27/2021-10:49:02] Iteration 14
[INFO-wrappers_bo-05/27/2021-10:50:58] Iteration 15
[INFO-wrappers_bo-05/27/2021-10:52:03] Iteration 16
[INFO-wrappers_bo-05/27/2021-10:53:21] Iteration 17
[INFO-wrappers_bo-05/27/2021-10:55:02] Iteration 18
[INFO-wrappers_bo-05/27/2021-10:57:02] Iteration 19
[INFO-wrappers_bo-05/